# Pytorch hands-on

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn, optim, Tensor
from sklearn import datasets
from tensorboardX import SummaryWriter

## Fitting Linear model

### Linear model module

In [ ]:
class LinearModel(nn.Module):
    def __init__(self, dim_input: int, dim_output: int): #, init_w: float=1.0):
        # TODO: see why this call is required
        super(LinearModel, self).__init__()

        # Initialize linear layer
        self.linear = nn.Linear(dim_input, dim_output)
#         self.linear.weight.data.uniform_(-init_w, init_w)
#         self.linear.bias.data.uniform_(-init_w, init_w)

    def forward(self, x: torch.Tensor):
        # Apply linear layer
        return self.linear(x)

### Loading dataset

Adapted from [scikit-learn example](https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html)

In [ ]:
def load_data():
    # Load the diabetes dataset
    diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True)

    # Use only one feature
    diabetes_X = diabetes_X[:, np.newaxis, 2]

    # Split the data into training/testing sets
    diabetes_X_train = diabetes_X[:-20]
    # diabetes_X_test = diabetes_X[-20:]

    # Split the targets into training/testing sets
    diabetes_y_train = diabetes_y[:-20]
    # diabetes_y_test = diabetes_y[-20:]

    print(diabetes_X_train.shape)
    print(diabetes_y_train.shape)

    x_m = np.mean(diabetes_X_train)
    x_s = np.std(diabetes_X_train)
    xs = (diabetes_X_train - x_m) / x_s

    y_m = np.mean(diabetes_y_train)
    y_s = np.std(diabetes_y_train)
    ys = (diabetes_y_train - y_m) / y_s

    return xs, ys

xs, ys = load_data()
plt.scatter(xs, ys)

### Fitting linear model by stochastic gradient

In [ ]:
def train(model: nn.Module, xs: np.ndarray, ys: np.ndarray, n_train_steps: int):
    # From numpy array to torch tensor
    xs = Tensor(xs)
    ys = Tensor(ys)
    
    # Loss function: MSE loss
    # https://pytorch.org/docs/stable/nn.html#mseloss
    loss = nn.MSELoss()
    
    # Optimizer: Adam
    # https://pytorch.org/docs/stable/optim.html#torch.optim.Adam
    optimizer = optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)

    for i in range(n_train_steps):
        # Reset gradient
        optimizer.zero_grad()
        
        # model.forward() is called
        # reshape(-1) converting shape from [422, 1] to [422]
        ys_pred = model(xs).reshape(-1)
        
        # Loss value
        loss_value = loss(ys_pred, ys)
        
        # Taking an optimization step
        # https://pytorch.org/docs/stable/optim.html#taking-an-optimization-step
        loss_value.backward()
        optimizer.step()
        
        if i % 200 == 0:
            print("Step={}, Loss = {}".format(i, loss_value))

### Prediction and plotting

In [ ]:
def pred_and_plot(model: nn.Module, xs: np.ndarray, ys: np.ndarray):
    x_min = np.min(xs)
    x_max = np.max(xs)
    xs_ = np.arange(x_min, x_max, 0.1)[:, np.newaxis]

    # Prediction
    ys_ = model(Tensor(xs_))
    
    # From torch Tensor to numpy array
    # https://pytorch.org/tutorials/beginner/former_torchies/tensor_tutorial.html#converting-torch-tensor-to-numpy-array
    ys_ = ys_.detach().numpy()
    
    plt.scatter(xs, ys)
    plt.plot(xs_, ys_)

### Putting together

In [ ]:
# Training
model = LinearModel(dim_input=1, dim_output=1)
xs, ys = load_data()
train(model, xs, ys, n_train_steps=1000)

# Prediction
pred_and_plot(model, xs, ys)

## Monitoring loss value during training with Tensorboard

In [ ]:
def train_with_tensorboard(model: nn.Module, xs: np.ndarray, ys: np.ndarray, n_train_steps: int):
    # Tensorboard
    writer = SummaryWriter("./log")

    # From numpy array to torch tensor
    xs = Tensor(xs)
    ys = Tensor(ys)
    
    # Loss function: MSE loss
    # https://pytorch.org/docs/stable/nn.html#mseloss
    loss = nn.MSELoss()
    
    # Optimizer: Adam
    # https://pytorch.org/docs/stable/optim.html#torch.optim.Adam
    optimizer = optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)

    for i in range(n_train_steps):
        # Reset gradient
        optimizer.zero_grad()
        
        # model.forward() is called
        # reshape(-1) converting shape from [422, 1] to [422]
        ys_pred = model(xs).reshape(-1)
        
        # Loss value
        loss_value = loss(ys_pred, ys)
        writer.add_scalar("reward_train", loss_value, i)
        
        # Taking an optimization step
        # https://pytorch.org/docs/stable/optim.html#taking-an-optimization-step
        loss_value.backward()
        optimizer.step()
        
        if i % 200 == 0:
            print("Step={}, Loss = {}".format(i, loss_value))

In [ ]:
# Training
model = LinearModel(dim_input=1, dim_output=1)
xs, ys = load_data()
train_with_tensorboard(model, xs, ys, n_train_steps=1000)

# Prediction
pred_and_plot(model, xs, ys)

## Saving/loading trained model

CNNとVAEは別のノートブックにした方が良さそう

## Convolutional neural network

## Variational autoencoder